In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils.testing import save_labeled_video

%load_ext autoreload
%autoreload 2

# Analyze results

## Read results DataFrame

In [ ]:
datasets_folder = '../datasets/'
results_folder = './checkpoints/checkpoints_360_640_mse/checkpoint_0010_results/'
df_out = pd.read_csv(os.path.join(results_folder, 'val_output.csv'))
df_out

## Analyze error

In [ ]:
image_size=(360, 640)

error = np.sqrt((image_size[1]*(df_out['x_true']-df_out['x_pred']))**2 + (image_size[0]*(df_out['y_true']-df_out['y_pred']))**2)
error = np.asarray(error)

In [ ]:
threshold = 5
print(f"Error smaller than {threshold} pixels: {100*len(np.nonzero(error<=threshold)[0])/len(error):.2g}%")

hist_range = 15
plt.hist(error.clip(max=hist_range), bins=np.arange(hist_range+1), density=True, align='left', rwidth=0.8)
plt.show()

## Save output video with heatmap

In [ ]:
save_labeled_video(filename_src=os.path.join(datasets_folder, 'dataset_finales_2020_en/video.mp4'),
                   filename_dst='finales_val_annotated_gt_heatmap.mp4',
                   position_df=df_out.loc[df_out['dataset_id']==0],
                   show_ground_truth=True,
                   heatmaps_folder=os.path.join(results_folder, 'heatmaps'))

In [ ]:
save_labeled_video(filename_src=os.path.join(datasets_folder, 'dataset_lluis/game5/video.mp4'),
                   filename_dst='game5_annotated_gt_heatmap.mp4',
                   position_df=df_out.loc[df_out['dataset_id']==1],
                   show_ground_truth=True,
                   heatmaps_folder=os.path.join(results_folder, 'heatmaps'))

In [ ]:
from torchvision.models import regnet_y_400mf, RegNet_Y_400MF_Weights

In [ ]:
model = regnet_y_400mf(num_classes=1000, weights=RegNet_Y_400MF_Weights.DEFAULT)

In [ ]:
import torch.nn as nn
model.fc = nn.Linear(in_features=440, out_features=2, bias=True)

In [ ]:
model